In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import mysql.connector
import time
import re

In [2]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="jobs-scraping.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user="root",
    password="123456789",
    database="job_scraping_database"
)

cursor = mydb.cursor()


In [21]:
# Define chrome_options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=./chromedriver.exe")

In [22]:
wd = webdriver.Chrome(options = chrome_options)

In [23]:
def monster_scraper(webpage, page_number):
    next_page = webpage + str(page_number)
    print(str(next_page))
    wd.get(next_page)
    time.sleep(3)
    try:
        # Locate the element containing job listings (replace with correct selector)
        job_listing_elem = wd.find_element(By.ID, "card-scroll-container")

        # Get the initial scroll height
        scroll_height = wd.execute_script("return arguments[0].scrollHeight;", job_listing_elem)

        while True:
            #Scroll down first
            wd.execute_script("arguments[0].scrollTop = arguments[1];", job_listing_elem)
            # Wait before scrolling up a little bit
            time.sleep(2)
            
            # Scroll up slightly using JavaScript
            wd.execute_script("arguments[0].scrollBy(0, -50);", job_listing_elem)

            # Wait for a brief moment before scrolling down again
            time.sleep(2)
            # Scroll down using JavaScript
            wd.execute_script("arguments[0].scrollTop = arguments[1];", job_listing_elem, scroll_height)

            # Wait for a brief moment to let the content load (you can adjust this value)
            time.sleep(3)

            # Get the new scroll height
            new_scroll_height = wd.execute_script("return arguments[0].scrollHeight;", job_listing_elem)

            if new_scroll_height == scroll_height:
                # Scrolling reached the end of the job listings
                break

            scroll_height = new_scroll_height  # Update the scroll height for the next iteration
            time.sleep(5)
    except NoSuchElementException:
        print("No More Data")
        
    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("div", class_ = "job-search-resultsstyle__JobCardWrap-sc-1wpt60k-4 gFeVEp")
    for job in jobs:
        #JobId 
        job_id = job.find("article", {"data-testid": "svx_jobCard"})["data-job-id"]
        #Check if the JobId already exists in the table
        select_query = "SELECT JobId FROM marketing_jobs WHERE JobId = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()
        
        if result is None:
            #Job Title
            job_title = job.find("a", {"data-testid": "jobTitle"}).text.strip()
            
            #Job Link
            job_link = "https:" + job.find("a", {"data-testid": "jobTitle"})["href"]
            
            #Company Name
            company_elem = job.find("span", {"data-testid": "company"})
            company_name = company_elem.text.strip() if company_elem else ""
            
            #Location
            location = job.find("span", {"data-testid": "jobDetailLocation"}).text.strip()
            
            #Salary
            salary_elem = job.find("div", {"data-tagtype-testid": "payTag"})
            salary = salary_elem.text.strip() if salary_elem else ""
            
            driver = webdriver.Chrome(options = chrome_options)
            driver.get(job_link)
            time.sleep(3)
            job_src = driver.page_source
            job_soup = bs(job_src, "html.parser")
            job_des_elem = job_soup.find("div", class_ = "descriptionstyles__DescriptionBody-sc-13ve12b-4 crOoVX")
            job_des = job_des_elem.text.strip() if job_des_elem else ""
            
            #Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO marketing_jobs (JobId, Title, Company, Location, Apply, Salary, JobDescription)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
            driver.quit()
        else:
            print("Skipping duplicate JobId:", job_id)
            
    print("Data updated")
    
    url = wd.current_url
    page_number_match = re.search(r'page=(\d+)', url)
    if page_number_match:
        page_number = int(page_number_match.group(1))
    else:
        page_number = 1  # Default to 1 if the page number is not found in the URL
    if page_number < 70:
        page_number += 1
        monster_scraper(webpage, page_number)
    else:
        cursor.close()
        mydb.close()
        wd.quit()
        

monster_scraper("https://www.monster.com/jobs/search?q=Entry+Level+Marketing&where=United+States&page=", 1)  

https://www.monster.com/jobs/search?q=Entry+Level+Marketing&where=United+States&page=1
Skipping duplicate JobId: 8cd8c474-d9c8-48d0-9c8c-f6bf89aecf80
Skipping duplicate JobId: 12a3622e-8acb-4c14-9f37-dbfaa7e081ee
Skipping duplicate JobId: 0f3b38c3-f1e5-4ef8-98fe-74c6e28e9494
Skipping duplicate JobId: ebb2cab6-da63-46c2-9889-c107df106e1b
Skipping duplicate JobId: 48baf304-5380-4a4b-8335-6e6869f92cf8
Skipping duplicate JobId: 40ec1300-89c3-4d6d-a01e-116c462a527a
Skipping duplicate JobId: 4364e467-46ca-4a11-b8b9-3cfccf2f3206
Skipping duplicate JobId: 08decbed-368a-413b-a947-43ad46a33028
Skipping duplicate JobId: b317a6a3-b32f-46f0-a008-fa1f0c879645
Skipping duplicate JobId: 2448499e-ae34-4ff6-a402-a98499c20949
Skipping duplicate JobId: 093f7581-1ca8-4e1e-87da-de513d8f65c2
Skipping duplicate JobId: 8552c286-6d02-402f-9b22-b5d130f1bccb
Skipping duplicate JobId: cf604b77-c2b3-4579-b492-f4defad75be3
Skipping duplicate JobId: 962df265-6f2f-4461-b0de-e628043d0308
Skipping duplicate JobId: 6271e